In [1]:
############################################################################################
# IMPORTS
############################################################################################
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
#import tensorflow.keras as keras

from tensorflow.keras import layers
from tensorflow.keras import regularizers


from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Flatten

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical



from IPython import display
from PIL import Image


import pathlib
import shutil
import tempfile
import concurrent


from numba import cuda 

import json
import glob





print(tf.__version__)
print(tf.config.list_physical_devices())
############################################################################################

2024-12-18 02:50:53.130719: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-18 02:50:53.244812: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-18 02:50:53.245415: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-18 02:50:53.384621: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-18 02:50:55.616407: W tensorflow/compiler/tf

2.16.2
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


2024-12-18 02:50:57.877661: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-18 02:50:58.099292: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [2]:
############################################################################################
# 1. CONSTANTS - PATHS
############################################################################################

DATA_FS251 = './data/iFood_2019'
CLASSES_FILE_NAME = '/formated_annot/classes_formated.csv'

TRAIN_INFO = '/annot/train_info.csv'
VAL_INFO = '/annot/val_info.csv'
TEST_INFO = '/annot/test_info.csv'

TRAIN_PICS_PATH = './data/iFood_2019/train_set/'
TEST_PICS_PATH = './data/iFood_2019/test_set/'
VAL_PICS_PATH = './data/iFood_2019/val_set/'

SEED = 111

#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [3]:
############################################################################################
# IMPORTING DATA
############################################################################################
df_classes = pd.read_csv(DATA_FS251 + CLASSES_FILE_NAME)
df_train = pd.read_csv(DATA_FS251 + TRAIN_INFO, names=['file_name', 'class_num'])
df_validate = pd.read_csv(DATA_FS251 + VAL_INFO, names=['file_name', 'class_num'])
df_test = pd.read_csv(DATA_FS251 + TEST_INFO, names=['file_name'])

df_train.head(3)

,file_name,class_num
0,train_101733.jpg,211
1,train_101734.jpg,211
2,train_101735.jpg,211


In [4]:
############################################################################################
# 2. CONSTANTS - MODEL
############################################################################################
training_history = dict()

N_TRAIN = len(df_train.iloc[:, 0])
EPOCHS = 20
BATCH_SIZE = 32
RESIZE_TO = (256, 256)
STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.01,
    decay_steps=STEPS_PER_EPOCH*1000,
    decay_rate=1,
    staircase=False)

In [5]:
############################################################################################
# import image files

train_pics = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PICS_PATH,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
#    image_size=RESIZE_TO,
    shuffle=False,
    seed=SEED,
    validation_split=None,
    subset=None,
#    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=True,
    data_format=None,
    verbose=True
)
val_pics = tf.keras.preprocessing.image_dataset_from_directory(
    VAL_PICS_PATH,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
#    image_size=RESIZE_TO,
    shuffle=False,
    seed=SEED,
    validation_split=None,
    subset=None,
#    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=True,
    data_format=None,
    verbose=True
)
test_pics = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PICS_PATH,
    labels=None,
    label_mode=None,
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
#    image_size=RESIZE_TO,
    shuffle=False,
    seed=SEED,
    validation_split=None,
    subset=None,
#    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=True,
    data_format=None,
    verbose=True
)

Found 118475 files belonging to 251 classes.
Found 11994 files belonging to 251 classes.
Found 118475 files.


In [6]:

#tf.keras.preprocessing.image.load_img(train_pics[155])
#plt.imshow(train_pics)

In [7]:
############################################################################################
# DEFINING THE MODEL
tf.keras.backend.clear_session()
model_supclass = tf.keras.models.Sequential([
        tf.keras.layers.Resizing(
        height = 256,
        width = 256,
        interpolation='bicubic',
        crop_to_aspect_ratio=False,
        pad_to_aspect_ratio=False,
        fill_mode='constant',
        fill_value=0.0,
        data_format=None),
# First conv. layers set 256p:
        tf.keras.layers.Conv2D(
                filters=250,
                kernel_size = (3,3),
                strides=(1, 1),
                padding='valid',
                data_format=None,
                dilation_rate=(1, 1),
                groups=1,
                activation='relu',
                use_bias=True,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                kernel_regularizer=None,
                bias_regularizer=None,
                activity_regularizer=None,
                kernel_constraint=None,
                bias_constraint=None
        ),
        tf.keras.layers.Conv2D(
                filters=250,
                kernel_size = (3,3),
                strides=(1, 1),
                padding='valid',
                data_format=None,
                dilation_rate=(1, 1),
                groups=1,
                activation='relu',
                use_bias=True,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                kernel_regularizer=None,
                bias_regularizer=None,
                activity_regularizer=None,
                kernel_constraint=None,
                bias_constraint=None
        ),
# Second conv. layers set 256->128:
        tf.keras.layers.MaxPooling2D(
                pool_size=(2, 2),
                strides=2,
                padding='valid',
                data_format=None,
                name='256_to_128_MaxPool'
        ),
        tf.keras.layers.Conv2D(
                filters=250,
                kernel_size = (3,3),
                strides=(1, 1),
                padding='valid',
                data_format=None,
                dilation_rate=(1, 1),
                groups=1,
                activation='relu',
                use_bias=True,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                kernel_regularizer=None,
                bias_regularizer=None,
                activity_regularizer=None,
                kernel_constraint=None,
                bias_constraint=None
        ),
        tf.keras.layers.Conv2D(
                filters=250,
                kernel_size = (3,3),
                strides=(1, 1),
                padding='valid',
                data_format=None,
                dilation_rate=(1, 1),
                groups=1,
                activation='relu',
                use_bias=True,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                kernel_regularizer=None,
                bias_regularizer=None,
                activity_regularizer=None,
                kernel_constraint=None,
                bias_constraint=None
        ),
# Third conv. layers set 128->64:
        tf.keras.layers.MaxPooling2D(
                pool_size=(2, 2),
                strides=2,
                padding='valid',
                data_format=None,
                name='128_to_64_MaxPool'
        ),
        tf.keras.layers.Conv2D(
                filters=251,
                kernel_size = (3,3),
                strides=(1, 1),
                padding='valid',
                data_format=None,
                dilation_rate=(1, 1),
                groups=1,
                activation='relu',
                use_bias=True,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                kernel_regularizer=None,
                bias_regularizer=None,
                activity_regularizer=None,
                kernel_constraint=None,
                bias_constraint=None
        ),
        tf.keras.layers.Conv2D(
                filters=251,
                kernel_size = (3,3),
                strides=(1, 1),
                padding='valid',
                data_format=None,
                dilation_rate=(1, 1),
                groups=1,
                activation='relu',
                use_bias=True,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                kernel_regularizer=None,
                bias_regularizer=None,
                activity_regularizer=None,
                kernel_constraint=None,
                bias_constraint=None
        ),
# Third conv. layers set 64->32:
        tf.keras.layers.MaxPooling2D(
                pool_size=(2, 2),
                strides=2,
                padding='valid',
                data_format=None,
                name='64_to_32_MaxPool'
        ),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(RESIZE_TO[0],kernel_initializer = 'uniform', activation=tf.keras.activations.sigmoid),
        tf.keras.layers.Dense(units=251,activation='softmax')
])



#model_supclass.summary()


In [8]:
############################################################################################
# COMPILING
############################################################################################
model_supclass.compile(
              optimizer = tf.keras.optimizers.Adam(learning_rate = 0.3),
              loss = tf.keras.losses.categorical_crossentropy,
              metrics = [tf.keras.metrics.categorical_accuracy] 
             )

#model_supclass.summary()

In [ ]:
############################################################################################
# FITTING
############################################################################################
with tf.device('/gpu:0'):
    model_history = model_supclass.fit(
        train_pics,
        epochs = EPOCHS, 
        batch_size = BATCH_SIZE, 
        validation_data = val_pics)

model_supclass.summary()

Epoch 1/20
   2/3703 ━━━━━━━━━━━━━━━━━━━━ 45:33:09 44s/step - categorical_accuracy: 0.2500 - loss: 3.6736    

In [ ]:
STOP_HERE

In [ ]:
############################################################################################
# EVALUATION
model_history.history.keys()
pd.DataFrame(model_history.history).plot()
plt.legend(bbox_to_anchor = [1, 1.02])
plt.plot(model_history.history['val_loss'])
plt.title('Validation Loss')
plt.xlabel('epochs')

# Print out the score
score = model.evaluate(X, y , batch_size=10, verbose=0)
print(score, model.metrics_names)

############################################################################################
# PREDICTION








y_hat=model.predict(new_X)
############################################################################################
# SAVING MODEL




model.save("model_moons.h5")
moons_model = load_model("model_moons.h5")
